# Dynamic BP-UCB: Time-Varying Demand

This notebook demonstrates the Dynamic BP-UCB algorithm for time-varying demand scenarios.

Based on:
> Singla, A., Santoni, M., Bartok, G., Mukerji, P., Meenen, M., and Krause, A. (2015). **Incentivizing Users for Balancing Bike Sharing Systems.** AAAI '15.

The algorithm extends BP-UCB to handle demand that varies across time periods (e.g., 2-hour slices for ride-sharing or bike-sharing).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../src")

from ucb_pricing import DynamicBPUCB, BPUCB

# Enable LaTeX rendering
plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif')

## Problem Setup

We divide a 24-hour day into 12 two-hour time slices. Each slice has:
- Different number of requests (following real-world demand patterns)
- Allocated budget proportional to expected demand

The Dynamic BP-UCB algorithm must balance exploration and exploitation while adapting to changing demand levels.

In [ ]:
# Parameters
c_min = 0.01
c_max = 1.0
K = 20

# Discretized prices
prices = np.linspace(c_min, c_max, K)

# Time slices represent 2-hour periods
time_labels = [
    "12am-2am", "2am-4am", "4am-6am", "6am-8am",
    "8am-10am", "10am-12pm", "12pm-2pm", "2pm-4pm",
    "4pm-6pm", "6pm-8pm", "8pm-10pm", "10pm-12am"
]

print(f"Price levels: K = {K}")
print(f"Price range: [{c_min}, {c_max}]")

## Demand Patterns

We simulate realistic demand with morning (8-10am) and evening (4-6pm) rush hours, typical of urban bike-sharing or ride-sharing systems.

In [ ]:
np.random.seed(42)

# Base demand with rush hour peaks
base_demand = np.array([20, 10, 15, 80, 150, 100, 90, 85, 140, 120, 70, 40])

# Total budget and proportional allocation
total_budget = 100
allocated_budgets = total_budget * (base_demand / base_demand.sum())

print("Time Slice Analysis:")
print(f"{'Time':<12} {'Demand':>8} {'Budget':>8}")
print("-" * 30)
for label, demand, budget in zip(time_labels, base_demand, allocated_budgets):
    print(f"{label:<12} {demand:>8} ${budget:>7.2f}")
print("-" * 30)
print(f"{'Total':<12} {sum(base_demand):>8} ${total_budget:>7.2f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(12)

ax1.bar(x, base_demand, color="steelblue")
ax1.set_xticks(x)
ax1.set_xticklabels(time_labels, rotation=45, ha="right")
ax1.set_ylabel("Number of Requests")
ax1.set_title("Demand Pattern Over 24 Hours")
ax1.grid(True, alpha=0.3, axis="y")

ax2.bar(x, allocated_budgets, color="coral")
ax2.set_xticks(x)
ax2.set_xticklabels(time_labels, rotation=45, ha="right")
ax2.set_ylabel(r"Allocated Budget (\$)")
ax2.set_title("Budget Allocation Over 24 Hours")
ax2.grid(True, alpha=0.3, axis="y")

plt.tight_layout()

## Running Dynamic BP-UCB

We start the algorithm at 8:00 AM (480 minutes from midnight) and run through a full 24-hour cycle.

In [ ]:
start_time = 480  # 8:00 AM

# Use actual demand (with some randomness)
requests_per_slice = np.random.poisson(base_demand)

dbp = DynamicBPUCB(prices=prices, n_slices=12)
result = dbp.run(
    start_time=start_time,
    allocated_budgets=allocated_budgets,
    requests_per_slice=requests_per_slice,
)

print(f"Total utility (workers hired): {result['total_utility']}")
print(f"Total requests: {sum(requests_per_slice)}")
print(f"Hire rate: {result['total_utility'] / sum(requests_per_slice) * 100:.1f}%")

## Price Selection Over Time

The algorithm learns and adapts prices as it processes requests throughout the day.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(result["price_offers"], alpha=0.7, linewidth=0.5, color="steelblue")
ax.axhline(np.mean(result["price_offers"]), color="red", linestyle="--", 
           label=f'Mean price = {np.mean(result["price_offers"]):.3f}')
ax.set_xlabel("Request Number")
ax.set_ylabel("Price Offered")
ax.set_title("Prices Offered Over Time")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()

## Utility per Time Slice

Comparing hired workers to requests in each time period.

In [ ]:
start_slice = dbp.get_time_slice(start_time)
reordered_labels = time_labels[start_slice:] + time_labels[:start_slice]
reordered_requests = np.concatenate([requests_per_slice[start_slice:], requests_per_slice[:start_slice]])

fig, ax = plt.subplots(figsize=(14, 5))

x = np.arange(12)
width = 0.35

ax.bar(x - width/2, reordered_requests, width, label="Requests", color="lightgray")
ax.bar(x + width/2, result["utilities_per_slice"], width, label="Hired", color="steelblue")
ax.set_xticks(x)
ax.set_xticklabels(reordered_labels, rotation=45, ha="right")
ax.set_ylabel("Count")
ax.set_title("Requests vs Workers Hired per Time Slice")
ax.legend()
ax.grid(True, alpha=0.3, axis="y")
plt.tight_layout()

## Multiple Simulations Analysis

Run multiple simulations to analyze average performance.

In [ ]:
n_simulations = 50
utilities = []
hire_rates = []

for sim in range(n_simulations):
    if sim % 10 == 0:
        print(f"Simulation {sim}/{n_simulations}")
    
    requests = np.random.poisson(base_demand)
    dbp_sim = DynamicBPUCB(prices=prices, n_slices=12)
    res = dbp_sim.run(
        start_time=start_time,
        allocated_budgets=allocated_budgets,
        requests_per_slice=requests,
    )
    utilities.append(res["total_utility"])
    hire_rates.append(res["total_utility"] / sum(requests))

print(f"\nResults over {n_simulations} simulations:")
print(f"Average utility: {np.mean(utilities):.1f} +/- {np.std(utilities):.1f}")
print(f"Average hire rate: {np.mean(hire_rates)*100:.1f}% +/- {np.std(hire_rates)*100:.1f}%")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.hist(utilities, bins=15, color="steelblue", alpha=0.7)
ax1.axvline(np.mean(utilities), color="red", linestyle="--", label=f"Mean = {np.mean(utilities):.1f}")
ax1.set_xlabel("Total Utility")
ax1.set_ylabel("Frequency")
ax1.set_title("Distribution of Utility")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.hist(np.array(hire_rates)*100, bins=15, color="coral", alpha=0.7)
ax2.axvline(np.mean(hire_rates)*100, color="red", linestyle="--", label=f"Mean = {np.mean(hire_rates)*100:.1f}\\%")
ax2.set_xlabel("Hire Rate (\\%)")
ax2.set_ylabel("Frequency")
ax2.set_title("Distribution of Hire Rate")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()

## Utility vs Total Budget

Analyze how utility scales with different budget levels.

In [ ]:
budgets = list(range(50, 250, 20))
avg_utilities = []
avg_hire_rates = []
n_sims = 20

for budget in budgets:
    utils = []
    rates = []
    alloc = budget * (base_demand / base_demand.sum())
    
    for _ in range(n_sims):
        requests = np.random.poisson(base_demand)
        dbp_sim = DynamicBPUCB(prices=prices, n_slices=12)
        res = dbp_sim.run(
            start_time=start_time,
            allocated_budgets=alloc,
            requests_per_slice=requests,
        )
        utils.append(res["total_utility"])
        rates.append(res["total_utility"] / sum(requests))
    
    avg_utilities.append(np.mean(utils))
    avg_hire_rates.append(np.mean(rates))

print("Done!")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(budgets, avg_utilities, 'g-o', markersize=6)
ax1.set_xlabel(r"Total Budget ($B$)")
ax1.set_ylabel("Average Utility")
ax1.set_title("Utility vs Budget")
ax1.grid(True, alpha=0.3)

ax2.plot(budgets, np.array(avg_hire_rates)*100, 'b-o', markersize=6)
ax2.set_xlabel(r"Total Budget ($B$)")
ax2.set_ylabel("Average Hire Rate (\\%)")
ax2.set_title("Hire Rate vs Budget")
ax2.grid(True, alpha=0.3)

plt.tight_layout()

## Comparison: Dynamic vs Static BP-UCB

Compare Dynamic BP-UCB (time-varying budgets) against standard BP-UCB (single budget pool).

In [ ]:
n_sims = 30
dynamic_utils = []
static_utils = []

for sim in range(n_sims):
    if sim % 10 == 0:
        print(f"Simulation {sim}/{n_sims}")
    
    requests = np.random.poisson(base_demand)
    total_requests = sum(requests)
    
    # Dynamic BP-UCB
    dbp_sim = DynamicBPUCB(prices=prices, n_slices=12)
    res_dynamic = dbp_sim.run(
        start_time=start_time,
        allocated_budgets=allocated_budgets,
        requests_per_slice=requests,
    )
    dynamic_utils.append(res_dynamic["total_utility"])
    
    # Static BP-UCB (single pool)
    bp_static = BPUCB(
        budget=total_budget,
        n_workers=total_requests,
        c_min=c_min,
        c_max=c_max,
        n_prices=K,
    )
    bids = np.random.uniform(c_min, c_max, total_requests)
    bp_static.set_bids(bids)
    res_static = bp_static.run()
    static_utils.append(res_static.utility)

print(f"\nDynamic BP-UCB: {np.mean(dynamic_utils):.1f} +/- {np.std(dynamic_utils):.1f}")
print(f"Static BP-UCB:  {np.mean(static_utils):.1f} +/- {np.std(static_utils):.1f}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(2)
means = [np.mean(dynamic_utils), np.mean(static_utils)]
stds = [np.std(dynamic_utils), np.std(static_utils)]

bars = ax.bar(x, means, yerr=stds, capsize=5, color=["steelblue", "coral"], alpha=0.8)
ax.set_xticks(x)
ax.set_xticklabels(["Dynamic BP-UCB", "Static BP-UCB"])
ax.set_ylabel("Average Utility")
ax.set_title("Dynamic vs Static BP-UCB Comparison")
ax.grid(True, alpha=0.3, axis="y")

for bar, mean in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f"{mean:.1f}", ha="center", va="bottom", fontsize=12)

plt.tight_layout()

## Arm Selection Distribution

Analyze which price arms are selected most frequently.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax.bar(prices, result["arm_counts"], width=0.04, color="steelblue")
ax.set_xlabel(r"Price ($p$)")
ax.set_ylabel("Selection Count")
ax.set_title("Arm Selection Frequency")
ax.grid(True, alpha=0.3, axis="y")
plt.tight_layout()

## Save Results Plot

Save the demand and hiring comparison plot for the README.

In [ ]:
# Run simulations to get average results
n_sims = 50
np.random.seed(42)

all_utilities = []
all_requests = []

for _ in range(n_sims):
    requests = np.random.poisson(base_demand)
    dbp_sim = DynamicBPUCB(prices=prices, n_slices=12)
    res = dbp_sim.run(
        start_time=start_time,
        allocated_budgets=allocated_budgets,
        requests_per_slice=requests,
    )
    all_utilities.append(res["utilities_per_slice"])
    all_requests.append(requests)

avg_utilities = np.mean(all_utilities, axis=0)
avg_requests = np.mean(all_requests, axis=0)

# Create and save figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(12)
width = 0.35
short_labels = [
    "12-2am", "2-4am", "4-6am", "6-8am",
    "8-10am", "10-12pm", "12-2pm", "2-4pm",
    "4-6pm", "6-8pm", "8-10pm", "10-12am"
]

# Left plot: Demand pattern
ax1.bar(x, base_demand, color="steelblue", alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels(short_labels, rotation=45, ha="right")
ax1.set_ylabel("Number of Requests")
ax1.set_title("Demand Pattern Over 24 Hours")
ax1.grid(True, alpha=0.3, axis="y")

# Right plot: Requests vs Hired
ax2.bar(x - width/2, avg_requests, width, label="Requests", color="lightgray")
ax2.bar(x + width/2, avg_utilities, width, label="Hired", color="green")
ax2.set_xticks(x)
ax2.set_xticklabels(short_labels, rotation=45, ha="right")
ax2.set_ylabel("Count")
ax2.set_title("Average Requests vs Workers Hired")
ax2.legend()
ax2.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.savefig("../assets/dynamic_bp_ucb_results.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"Saved to assets/dynamic_bp_ucb_results.png")
print(f"Average utility: {sum(avg_utilities):.0f}")
print(f"Average requests: {sum(avg_requests):.0f}")
print(f"Hire rate: {sum(avg_utilities)/sum(avg_requests)*100:.1f}%")